# Qumulo API Activity Data exmaples

In [1]:
from qumulo.rest_client import RestClient

In [3]:
# change these to your Qumulo API cluster and credentials
rc = RestClient('<qumulo-cluster>', 8000)
rc.login('<login>', '<password>');

### make the current activity api call that we'll be using below

In [20]:
activity_data = rc.analytics.current_activity_get()

### Show all of types of activity data

In [21]:
list(set([d['type'] for d in activity_data['entries']]))

[u'file-iops-write',
 u'file-throughput-read',
 u'file-iops-read',
 u'metadata-iops-read',
 u'metadata-iops-write',
 u'file-throughput-write']

### Sum up the rates by activity type

In [22]:
types_data = {}
for d in activity_data['entries']:
    if d['type'] not in types_data:
        types_data[d['type']] = 0
    types_data[d['type']] += d['rate']
for t, val in types_data.iteritems():
    print("%22s - %.1f" % (t, val))

       file-iops-write - 4.9
  file-throughput-read - 9147.7
        file-iops-read - 0.7
    metadata-iops-read - 13.0
   metadata-iops-write - 1.6
 file-throughput-write - 254429.9


### Report clients by throughput from highest to lowest

In [23]:
ip_data = {}
for d in activity_data['entries']:
    if 'throughput' in d['type']:
        if d['ip'] not in ip_data:
            ip_data[d['ip']] = 0
        ip_data[d['ip']] += d['rate']
for ip, val in sorted(ip_data.items(), key=lambda x: x[1], reverse=True):
    print("%22s - %.1f" % (ip, val))

        10.220.246.245 - 238045.9
        10.220.246.241 - 25531.7


### Resolve ids to paths

In [24]:
batch_size = 1000
ids_to_paths = {}

ids = list(set([d['id'] for d in activity_data['entries']]))

for offset in range(0, len(ids), batch_size):
    resolve_data = rc.fs.resolve_paths(ids[offset:offset+batch_size])
    for id_path in resolve_data:
        ids_to_paths[id_path['id']] = id_path['path']

print("Resolved %s ids to paths." % len(ids_to_paths))

Resolved 433 ids to paths.


### Resolve ips to hostnames

In [25]:
batch_size = 100
ips_to_names = {}

ips = list(set([d['ip'] for d in activity_data['entries']]))

for offset in range(0, len(ips), batch_size):
    resolve_data = rc.dns.resolve_ips_to_names(ips[offset:offset+batch_size])
    for d in resolve_data:
        ips_to_names[d['ip_address']] = d['hostname']

print("Resolved %s ips to hosts." % len(ips_to_names))

Resolved 2 ips to hosts.


### Summarize the most active directories by total throughput

In [26]:
directory_levels = 1
path_data = {}

for d in activity_data['entries']:
    path = ids_to_paths[d['id']]
    if path == '':
        path = '/'
    path = '/'.join(path.split('/')[0:directory_levels+1])
    if 'throughput' in d['type']:
        if path not in path_data:
            path_data[path] = 0
        path_data[path] += d['rate']

for path, val in sorted(path_data.items(), key=lambda x: x[1], reverse=True):
    print("%32s - %.1f" % (path, val))

                       /query_db - 194764.8
                               / - 36659.2
                  /elasticsearch - 14540.8
                      /databases - 6690.1
                    /power-tools - 6485.3
              /qumulo-audit-logs - 4437.3


### Summarize the most active clients by total throughput

In [27]:
ip_data = {}

for d in activity_data['entries']:
    if 'throughput' in d['type']:
        if d['ip'] not in path_data:
            ip_data[d['ip']] = 0
        ip_data[d['ip']] += d['rate']

for ip, val in sorted(ip_data.items(), key=lambda x: x[1], reverse=True):
    print("%20s (%s)  - %.1f" % (ips_to_names[ip], ip, val))

mc-25.eng.qumulo.com (10.220.246.241)  - 7304.5
mc-29.eng.qumulo.com (10.220.246.245)  - 341.3
